## Imports

In [45]:
path='data/'
#import colab_env
import os
import datetime
from time import strptime
from dotenv import load_dotenv
load_dotenv(dotenv_path='')
# The API
from amadeus import Client, ResponseError

# This library can extract dates from text
import dateutil.parser as dparser

# We'll need the following nltk packages
import nltk
nltk.download('punkt')
nltk.download('wordnet')

# We'll use lemmas instead of stems.
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# We will tag the parts of the speech 
from nltk.tag import pos_tag

# Get the stop words
from nltk.corpus import stopwords
nltk.download('stopwords')

# We'll need json to open a file of intents
import json

# We will need to save the model
import pickle

# Some helper libraries
import numpy as np
import pandas as pd
import random
import re

# We'll need these to actually create a machine learning model.
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD, Adagrad, Adam, Adadelta

# We may need to try out a grid search
from sklearn.model_selection import GridSearchCV

# We'll need this for the in-code tagging of words
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.matcher import Matcher

from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from chatterbot.trainers import ChatterBotCorpusTrainer

[nltk_data] Downloading package punkt to /home/karl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/karl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/karl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Training Data

In [46]:
# Greetings
hellos = [['Hello!','Hi there'],
          ['Hi there','Hello!'],
          ['Greetings','Hey'],
          ['Hey','Oh hey!'],
          ['Good morning','Good morning'],
          ['Good evening','Good evening'],
          ['Hey there!','Well hello there']
         ]

# Farewells
byes = [['Bye','Goodbye!'],
        ['See ya','Good bye!'],
        ['Have a good night','You as well, goodbye!'],
        ['Farewell','Later!'],
        ['Goodbye!', 'Bye for now!']
       ]

In [47]:
convos = [
          ["I would like to book a flight.",
           "I can help you with that. Where were you thinking of traveling to?"
          ],
          ['I want to buy a plane ticket.',
           'I can help you make your reservation! Can you provide some more details?'
          ],
          ['I need to make a plane reservation.',
           'I can book your trip right now. What city are you flying to, from?'
          ],
          ['I want to take a vacation',
           'I can help you with that! What were you thinking?'
          ],
          ['I need a goddamn vacation',
           'Where do you come from, where do you go? Where are you coming from, Cotton Eyed Joe?'
          ],
          ['I want to book a flight',
           'For sure! Can give me some details, like where you want to go and when?'
          ],
          ['Book me a flight',
           "Definitely! What sort of trip were you thinking of?"
          ],
          ["Can I book a plane ticket?",
           "Absolutely! Can you provide some more details for your trip?"
          ],
          ["I would like to book a trip",
           "Awesome, what sort of trip were you thinking?"              
          ]
         ]

## Training the Bot

In [48]:
# We will first train the bot with the lists of data we've given it above.
chatbot = ChatBot('Traveller',logic_adapters=["chatterbot.logic.BestMatch"])
travel_listtrainer = ListTrainer(chatbot)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/karl/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/karl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/karl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:
for convo in convos:
    travel_listtrainer.train(convo)

for hi in hellos:
    travel_listtrainer.train(hi)

for bye in byes:
    travel_listtrainer.train(bye)

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%


In [50]:
# Next we will need to train in with the corpus.
travel_corpustrainer = ChatterBotCorpusTrainer(chatbot)
travel_corpustrainer.train("chatterbot.corpus.english")

Training ai.yml: [###                 ] 16%

/home/karl/Documents/chatbot/ChatBot_1000ml_p7/cb_env/lib/python3.8/site-packages/chatterbot/corpus.py:38: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(data_file)


Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


## Functions

### Matcher Function
Uses SpaCy pattern matching to discover information needed for booking.

In [51]:
def depart_date_matcher(tokenized):
    matcher = Matcher(nlp.vocab)
    matcher.add("DateFromFinder", None, [{'LOWER':'leave'}, {'LOWER':'on'}, {"ENT_TYPE": "DATE"}],
                                        [{'LOWER':'leaving'}, {'LOWER':'on'}, {"ENT_TYPE": "DATE"}],
                                        [{'LOWER':'leave'}, {"ENT_TYPE": "DATE"}],
                                        [{'LOWER':'leaving'}, {"ENT_TYPE": "DATE"}],
                                        [{'LOWER':'go'}, {'LOWER':'on'}, {"ENT_TYPE": "DATE"}],
                                        [{'LOWER':'on'}, {"ENT_TYPE": "DATE"}], 
                                        [{'LOWER':'from'}, {"ENT_TYPE": "DATE"}])
    return matcher(tokenized)

    
def return_date_matcher(tokenized):
    matcher = Matcher(nlp.vocab)
    matcher.add("DateToFinder", None, [{'LOWER':'return'},{'LOWER':'on'}, {"ENT_TYPE": "DATE"}],
                                      [{'LOWER':'returning'},{'LOWER':'on'}, {"ENT_TYPE": "DATE"}],
                                      [{'LOWER':'return'},{"ENT_TYPE": "DATE"}],
                                      [{'LOWER':'returning'}, {"ENT_TYPE": "DATE"}],
                                      [{'LOWER':'back'},{'LOWER':'on'}, {"ENT_TYPE": "DATE"}],
                                      [{'LOWER':'back'}, {"ENT_TYPE": "DATE"}],
                                      [{'LOWER':'for'}, {"ENT_TYPE": "DATE"}],
                                      [{'LOWER':'until'}, {"ENT_TYPE": "DATE"}],
                                      [{'LOWER':'to'}, {"ENT_TYPE": "DATE"}])
    return matcher(tokenized)
   

def origin_matcher(tokenized):
    matcher = Matcher(nlp.vocab)
    matcher.add("OriginFinder", None, [{'LOWER':'from'},{'ENT_TYPE':'GPE'}],
                                      [{'LOWER':'of'},{'ENT_TYPE':'GPE'}],
                                      [{'LOWER':'leaving'},{'ENT_TYPE':'GPE'}],
                                      [{'LOWER':'leave'},{'ENT_TYPE':'GPE'}])
    return matcher(tokenized)

def dest_matcher(tokenized):
    matcher = Matcher(nlp.vocab)
    matcher.add("DestinationFinder", None, [{'LOWER':'to'},{'ENT_TYPE':'GPE'}],
                                           [{'LOWER':'visit'},{'ENT_TYPE':'GPE'}],
                                           [{'LOWER':'see'},{'ENT_TYPE':'GPE'}],
                                           [{'LOWER':'out'},{'ENT_TYPE':'GPE'}],
                                           [{'LOWER':'travel'},{'ENT_TYPE':'GPE'}])
    return matcher(tokenized)

def budget_matcher(tokenized):
    matcher = Matcher(nlp.vocab)
    matcher.add('BudgetFinder', None, [{'LOWER':'of'},{'IS_DIGIT':True}],
                                      [{'LOWER':'spend'},{'IS_DIGIT':True}],
                                      [{'LOWER':'under'},{'IS_DIGIT':True}],
                                      [{'LOWER':'than'},{'IS_DIGIT':True}],
                                      [{'ENT_TYPE':'MONEY'}],
                                      [{'ORTH':'$'},{'IS_DIGIT':True}])
    return matcher(tokenized)

### Looker Function
This is the function which uses the matcher function and returns what, if any, key words are in the user message.

In [52]:
def entity_looker(message):
    '''
    This function looks at a message from the user and seeks for matches in dates, locations and budget
    and outputs a value of True if they are contained in the message. 
    '''
    
    # Tokenize the sentence for entity matching.
    tokens = nlp(message)
    # This portion uses the boolean nature of an empty list. If the list is empty, we get False, else True
    # Date From Match
    DF_match = not not depart_date_matcher(tokens)
    
    # Date To Match
    DT_match = not not return_date_matcher(tokens)
    
    # Origin Match
    O_match = not not origin_matcher(tokens)
    
    # Destination Match
    D_match = not not dest_matcher(tokens)
    
    # Budget Match
    B_match = not not budget_matcher(tokens)
    
    # This returns a tuple of the Trues/Falses, so the functions know what they are looking for specifically.
    return (DF_match, DT_match, O_match, D_match, B_match)

### Picker Function
Uses the looker function. If any matches found, this uses the boolean value to actually do something with the match.

In [53]:
def entity_picker(message):
    '''
    This function takes the match information, and calls on the functions necessary to extract the info
    from the message. It then converts the entities that it picked out to an API readable form.
    It then returns a set of the entities. If there were no matches, it returns None.
    '''
    
    matches = entity_looker(message)
    from_date = to_date = origin = dest = budget_int = ''
    
    # From Date match
    if matches[0]:
        # Get the from date (comes back as DateTime)
        depart_date_dt = get_depart_date(message)
        # Convert dates for API
        depart_date = convert_date(depart_date_dt)
    
    # To Date match
    if matches[1]:
        # Get the from date (comes back as DateTime)
        return_date_dt = get_return_date(message)
        # Convert dates for API
        return_date = convert_date(return_date_dt)
        
    # Origin Location match
    if matches[2]:
        # Get the origin location
        origin = get_origin(message)
    
    # Destination Location match
    if matches[3]:
        # get the destination location
        dest = get_dest(message)
    
    # Budget match
    if matches[4]:
        # get the budget
        budget = get_budget(message)
        # convert to int
        budget_int = int(budget)
    
    return (depart_date, return_date, origin, dest, budget_int)

### Depart Date Extractor
This function returns the depart date in a datetime format.

In [54]:
def get_depart_date(message):
    '''
    This function is designed to extract a datetime object from the departure date, called "from_date". 
    If there are two dates in the message, I will _assume_ that the first date is the departure date, and the
    second is the return date. 
    '''
    # Tokenize
    tokens = nlp(message)
    # Extract entities
    match = depart_date_matcher(tokens)
    desired_date = str(tokens[match[0][1]:match[0][2]+1])

    # Next, we will look to see if someone used the words "tomorrow", "next week" or "today"
    today = datetime.datetime.today()
    
    if any(day in desired_date for day in ['today','tomorrow','next week']):
        depart_date = word_dates(desired_date, today)
    else: 
        depart_date = dparser.parse(desired_date,fuzzy=True)
        
    return depart_date

### Return Date Extractor
Does the same as above, but for the return date.

In [55]:
def get_return_date(message):
    '''
    This function is designed to extract a datetime object from the Return date, called "to_date". 
    If there are two dates in the message, I will _assume_ that the first date is the departure date, and the
    second is the return date. 
    '''
    # Tokenize
    tokens = nlp(message)
    # Extract entities
    match = return_date_matcher(tokens)
    desired_date = str(tokens[match[0][1]:match[0][2]+1])

    # Next, we will look to see if someone used the words "tomorrow", "next week" or "today"
    today = datetime.datetime.today()
    
    if any(day in desired_date for day in ['today','tomorrow','next week']):
        return_date = word_dates(desired_date, today)
    else: 
        return_date = dparser.parse(desired_date,fuzzy=True)
        
    return return_date

### Get Word Dates
For times like 'today', 'tomorrow' and 'next week', we need a way to turn them into actual dates.

In [16]:
def word_dates(date_text, today):
    if 'today' in date_text:
        date = today
    elif 'tomorrow' in date_text:
        date = today + datetime.timedelta(days=1)
    elif 'next week' in date_text:
        date = today + datetime.timedelta(days=7)
    else:
        date = None
    return date

### Date Converter
The API only takes dates in a particular string format. Let's make a function that does that.

In [17]:
def convert_date(dtime):
    '''
    This function converts a date time object to a month, day, year. This is required for the API
    '''
    year = dtime.year
    month = str(dtime.month) if int(dtime.month) >9 else '0'+str(dtime.month)
    day = str(dtime.day) if int(dtime.day)>9 else '0'+str(dtime.day)
    formatted_time = f"{year}-{month}-{day}"
    
    return formatted_time

### Get Origin Location
Extracts the location of origin for the flight.

In [18]:
def get_origin(message):
    
    # Tokenize
    tokens = nlp(message)
    # Extract entities
    entities = [(X.label_, X.text) for X in tokens.ents]
    # get rid of anything not a date
    loc_ent = [x for x in entities if x[0]=='GPE']
    locs = [x[1].lower() for x in loc_ent]
    
    if len(locs)>1:
        # If there is more than one location in the string, we just need to figure out what order those locations
        # are in, in the sentence, which this logic does.
        if dest_matcher(tokens)[0][2] > origin_matcher(tokens)[0][2]:
            origin = locs[0]
        else:
            origin = locs[1]
    else:
        origin = locs[0]
        
    return origin

### Get Destination Location
Extracts the destination for the flight

In [19]:
def get_dest(message):
    
    # Tokenize
    tokens = nlp(message)
    # Extract entities
    entities = [(X.label_, X.text) for X in tokens.ents]
    # get rid of anything not a date
    loc_ent = [x for x in entities if x[0]=='GPE']
    locs = [x[1].lower() for x in loc_ent]
    
    if len(locs)>1:
        # If there is more than one location in the string, we just need to figure out what order those locations
        # are in, in the sentence, which this logic does.
        if dest_matcher(tokens)[0][2] > origin_matcher(tokens)[0][2]:
            dest = locs[1]
        else:
            dest = locs[0]
    else:
        dest = locs[0]
        
    return dest

### Budget Extractor
Extracts the budget from the text.

In [20]:
def get_budget(message):
    no_punct = re.sub(r'[^\w\s$]','',message)
    tokens = nlp(no_punct)
    match = budget_matcher(tokens)
    try:
        # Near the position of the first match, scan for numbers. The first number present is the budget.
        budget_ls = re.findall(r'[\d\.\d]+', str(tokens[match[0][1]:]))
        budget = budget_ls[0]
    except IndexError:
        budget = 0

    return budget

In [21]:
get_budget('$2000')

'2000'

### IATA code translator
Turns a lowercase city name into a corresponding IATA code for airports. 

In [22]:
# Read in IATA code csv
city_code = pd.read_csv('Airports.csv')
# make everything lowercase to account for user 
city_code['Location'] = city_code['Location'].str.lower()
# Make the location the index
city_code = city_code.groupby('Location').max()
# Turn into dictionary where the cities are the keys and codes are the values.
code_dict = city_code.to_dict()['Code']

def iata_code_lookup(city, codes, origin_city=True):
    '''
    This function takes in a city name as well as a dictionary with city names and their IATA codes. It then checks whether this is the origin city or not.
    If the city is in the dictionary, it will output the code. Otherwise, it will output an empty string and an error message that depends on whether or not the city was the origin or destination.
    '''
    try: 
        iata_code = codes[city]
    except KeyError:
        iata_code = ''
        if origin_city:
            print(f'Unfortunately there are no flights leaving {city} at this time.')
            return None
        else:
            print(f'Unfortunately there are no flights to {city} at this time.')
            return None
    return iata_code

### Make API Call
Returns result of flight booker API. 

In [23]:
def get_flight(flight_info, codes):
    amadeus = Client(
        client_id=os.getenv("AMADEUS_API_KEY"),
        client_secret=os.getenv("AMADEUS_SECRET")
    )

    try:
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode = iata_code_lookup(flight_info['origin_loc'], codes, origin_city=True),
            destinationLocationCode = iata_code_lookup(flight_info['dest_loc'], codes, origin_city=False),
            departureDate = flight_info['depart_date'],
            returnDate = flight_info['return_date'],
            maxPrice = flight_info['budget'],
            currencyCode='CAD',
            adults=1)
        result = f'''Your flight has been booked! Here are the details, tickets will be sent to you by email.
        Departure Time: {response.data[0]['itineraries'][0]['segments'][0]['departure']['at']}
        Arrival Time: {response.data[0]['itineraries'][0]['segments'][0]['arrival']['at']}
        Total Price: {response.data[0]['price']['total']}
        Flight Class: {response.data[0]['travelerPricings'][0]['fareDetailsBySegment'][0]['cabin']}
        '''
    except ResponseError as error:
        result = error
    return result

## The Bot

In [24]:
flight_info = {'depart_date':0,
               'return_date':0,
               'origin_loc':0,
               'dest_loc':0,
               'budget':0
              }

def Traveler(message):
    # First, if the message contains a confirmation, we should continue with the API booking
    if ('confirmed' in message) or ('Confirmed' in message):
        return get_flight(flight_info, code_dict)
    # See what matches we have:
    values = entity_picker(message)
    
    if values[0] != '':
        flight_info['depart_date'] = values[0]
        
    if values[1] != '':
        flight_info['return_date'] = values[1]
        
    if values[2] != '':
        flight_info['origin_loc'] = values[2]
        
    if values[3] != '':
        flight_info['dest_loc'] = values[3]
        
    if values[4] != '':
        flight_info['budget'] = values[4]
    
    constraint_bool = [vals!=0 for vals in flight_info.values()]
    if (any(constraint_bool)) and not (all(constraint_bool)):
        if flight_info['dest_loc'] == 0:
            response_list_dest_loc = ['Where did you want to go?', 
                                      'Where were you thinking of travelling to?', 
                                      'What is your desired destination?']
            return random.choice(response_list_dest_loc)
        if flight_info['origin_loc'] == 0: 
            response_list_origin_loc = ['Where are you leaving from?', 
                                        'What city are you flying out of?', 
                                        'Where will you be flying from?']
            return random.choice(response_list_origin_loc)
        if flight_info['depart_date'] == 0:
            response_list_depart_date = ['When did you want to leave?', 
                                         'What dates were you thinking?', 
                                         'When is your travel date?', 
                                         'For what days should I book the flight?']
            return random.choice(response_list_depart_date)
        if flight_info['return_date'] == 0:
            response_list_return_date = ['When did you want to come back?', 
                                         'When do you need to return?', 
                                         'What should I set for return date?']
            return random.choice(response_list_return_date)
        if flight_info['budget'] == 0:
            response_list_budget = ['How much are you willing to spend on this trip?', 
                                    "What's your budget for this trip?", 
                                    "Do you have a price limit for the trip?"]
            return random.choice(response_list_budget)
    
    elif all(constraint_bool):
        finalize= f"Awesome! Looks like I have everything I need to book your flight! Let me just confirm with you. You are booking a flight to {flight_info['dest_loc'].upper()}, and you are flying from {flight_info['origin_loc'].upper()}. You will be leaving on {flight_info['depart_date']} and returning on {flight_info['return_date']}. Finally, your budget is ${flight_info['budget']}. Is all this information correct? If yes, please type 'Confirmed'. If this is not correct, please provide the correct info in detail."
        return finalize
    
    else:
        return chatbot.get_response(message)

# API Practice

In [4]:
amadeus = Client(
        client_id=os.getenv("AMADEUS_API_KEY"),
        client_secret=os.getenv("AMADEUS_SECRET")
    )
response = amadeus.shopping.flight_offers_search.get(
            originLocationCode = 'YYZ',
            destinationLocationCode = 'YVR',
            departureDate = '2020-07-15',
            returnDate = '2020-07-24',
            maxPrice = 2000,
            currencyCode='CAD',
            adults=1)

In [41]:
for i, offer in enumerate(response.data[:5]):
    to_dep = dparser.parse(offer['itineraries'][0]['segments'][0]['departure']['at'],fuzzy=True)
    to_arr = dparser.parse(offer['itineraries'][0]['segments'][0]['arrival']['at'],fuzzy=True)
    re_dep = dparser.parse(offer['itineraries'][1]['segments'][0]['departure']['at'],fuzzy=True)
    re_arr = dparser.parse(offer['itineraries'][1]['segments'][0]['arrival']['at'],fuzzy=True)
    print(f'''
\033[1mOption {i+1}\033[0m
Departure Time:        {to_dep.hour}:{to_dep.minute if to_dep.minute>9 else str(0)+str(to_dep.minute)} on {to_dep.year}-{to_dep.month}-{to_dep.day}
Arrival Time:          {to_arr.hour}:{to_arr.minute if to_arr.minute>9 else str(0)+str(to_arr.minute)} on {to_arr.year}-{to_arr.month}-{to_arr.day}
Flight Duration:       {offer['itineraries'][0]['duration'].strip('PT').lower()}
Return Departure Time: {re_dep.hour}:{re_dep.minute if re_dep.minute>9 else str(0)+str(re_dep.minute)} on {re_dep.year}-{re_dep.month}-{re_dep.day}
Return Arrival Time:   {re_arr.hour}:{re_arr.minute if re_arr.minute>9 else str(0)+str(re_arr.minute)} on {re_arr.year}-{re_arr.month}-{re_arr.day}
Flight Duration:       {offer['itineraries'][1]['duration'].strip('PT').lower()}
Total Price:           ${offer['price']['total']}
Flight Class:          {offer['travelerPricings'][0]['fareDetailsBySegment'][0]['cabin']}
''')


Option 1
Departure Time:        19:45 on 2020-7-15
Arrival Time:          21:55 on 2020-7-15
Flight Duration:       5h10m
Return Departure Time: 11:00 on 2020-7-24
Return Arrival Time:   18:35 on 2020-7-24
Flight Duration:       4h35m
Total Price:           $422.88
Flight Class:          ECONOMY


Option 2
Departure Time:        19:45 on 2020-7-15
Arrival Time:          21:55 on 2020-7-15
Flight Duration:       5h10m
Return Departure Time: 23:00 on 2020-7-24
Return Arrival Time:   6:35 on 2020-7-25
Flight Duration:       4h35m
Total Price:           $422.88
Flight Class:          ECONOMY


Option 3
Departure Time:        15:00 on 2020-7-15
Arrival Time:          17:06 on 2020-7-15
Flight Duration:       5h6m
Return Departure Time: 21:50 on 2020-7-24
Return Arrival Time:   22:49 on 2020-7-24
Flight Duration:       6h24m
Total Price:           $873.09
Flight Class:          ECONOMY


Option 4
Departure Time:        15:00 on 2020-7-15
Arrival Time:          17:06 on 2020-7-15
Flight Dura

In [20]:
response.data[0]

{'type': 'flight-offer',
 'id': '1',
 'source': 'GDS',
 'instantTicketingRequired': False,
 'nonHomogeneous': False,
 'oneWay': False,
 'lastTicketingDate': '2020-07-16',
 'numberOfBookableSeats': 9,
 'itineraries': [{'duration': 'PT5H10M',
   'segments': [{'departure': {'iataCode': 'YYZ',
      'terminal': '3',
      'at': '2020-07-15T19:45:00'},
     'arrival': {'iataCode': 'YVR',
      'terminal': 'M',
      'at': '2020-07-15T21:55:00'},
     'carrierCode': 'TS',
     'number': '932',
     'aircraft': {'code': '73H'},
     'operating': {'carrierCode': 'TS'},
     'duration': 'PT5H10M',
     'id': '16',
     'numberOfStops': 0,
     'blacklistedInEU': False}]},
  {'duration': 'PT4H35M',
   'segments': [{'departure': {'iataCode': 'YVR',
      'terminal': 'M',
      'at': '2020-07-24T11:00:00'},
     'arrival': {'iataCode': 'YYZ',
      'terminal': '3',
      'at': '2020-07-24T18:35:00'},
     'carrierCode': 'TS',
     'number': '931',
     'aircraft': {'code': '73H'},
     'operating'

In [13]:
dep = dparser.parse('2020-07-15T18:06:05',fuzzy=True)

In [19]:
f'{dep.hour}:{dep.minute if dep.minute>9 else str(0)+str(dep.minute)} on {dep.year}-{dep.month}-{dep.day}'

'18:06 on 2020-7-15'

In [24]:
{'type': 'flight-offer',
 'id': '1',
 'source': 'GDS',
 'instantTicketingRequired': False,
 'nonHomogeneous': False,
 'oneWay': False,
 'lastTicketingDate': '2020-07-16',
 'numberOfBookableSeats': 9,
 'itineraries': [{'duration': 'PT5H10M',
                  'segments': [{'departure': {'iataCode': 'YYZ',
                                              'terminal': '3',
                                              'at': '2020-07-15T19:45:00'},
                                'arrival': {'iataCode': 'YVR',
                                            'terminal': 'M',
                                            'at': '2020-07-15T21:55:00'},
                                'carrierCode': 'TS',
                                'number': '932',
                                'aircraft': {'code': '73H'},
                                'operating': {'carrierCode': 'TS'},
                                'duration': 'PT5H10M',
                                'id': '16',
                                'numberOfStops': 0,
                                'blacklistedInEU': False}]},
                 {'duration': 'PT4H35M',
                  'segments': [{'departure': {'iataCode': 'YVR',
                                              'terminal': 'M',
                                              'at': '2020-07-24T11:00:00'},
                                'arrival': {'iataCode': 'YYZ',
                                            'terminal': '3',
                                            'at': '2020-07-24T18:35:00'},
                                'carrierCode': 'TS',
                                'number': '931',
                                'aircraft': {'code': '73H'},
                                'operating': {'carrierCode': 'TS'},
                                'duration': 'PT4H35M',
                                'id': '73',
                                'numberOfStops': 0,
                                'blacklistedInEU': False}]}],
 
 'price': {'currency': 'CAD',
           'total': '422.88',
           'base': '280.00',
           'fees': [{'amount': '0.00', 'type': 'SUPPLIER'},
                    {'amount': '0.00', 'type': 'TICKETING'}],
           'grandTotal': '422.88'},
 'pricingOptions': {'fareType': ['PUBLISHED'],
                    'includedCheckedBagsOnly': False},
 'validatingAirlineCodes': ['TS'],
 'travelerPricings': [{'travelerId': '1',
                       'fareOption': 'STANDARD',
                       'travelerType': 'ADULT',
                       'price': {'currency': 'CAD', 'total': '422.88', 'base': '280.00'},
                       'fareDetailsBySegment': [{'segmentId': '16',
                                                 'cabin': 'ECONOMY',
                                                 'fareBasis': 'QKBGT1DO',
                                                 'brandedFare': 'BGT',
                                                 'class': 'Q',
                                                 'includedCheckedBags': {'quantity': 0}},
                                                {'segmentId': '73',
                                                 'cabin': 'ECONOMY',
                                                 'fareBasis': 'QKBGT1DO',
                                                 'brandedFare': 'BGT',
                                                 'class': 'Q',
                                                 'includedCheckedBags': {'quantity': 0}}]}]}

{'type': 'flight-offer',
 'id': '1',
 'source': 'GDS',
 'instantTicketingRequired': False,
 'nonHomogeneous': False,
 'oneWay': False,
 'lastTicketingDate': '2020-07-16',
 'numberOfBookableSeats': 9,
 'itineraries': [{'duration': 'PT5H10M',
   'segments': [{'departure': {'iataCode': 'YYZ',
      'terminal': '3',
      'at': '2020-07-15T19:45:00'},
     'arrival': {'iataCode': 'YVR',
      'terminal': 'M',
      'at': '2020-07-15T21:55:00'},
     'carrierCode': 'TS',
     'number': '932',
     'aircraft': {'code': '73H'},
     'operating': {'carrierCode': 'TS'},
     'duration': 'PT5H10M',
     'id': '16',
     'numberOfStops': 0,
     'blacklistedInEU': False}]},
  {'duration': 'PT4H35M',
   'segments': [{'departure': {'iataCode': 'YVR',
      'terminal': 'M',
      'at': '2020-07-24T11:00:00'},
     'arrival': {'iataCode': 'YYZ',
      'terminal': '3',
      'at': '2020-07-24T18:35:00'},
     'carrierCode': 'TS',
     'number': '931',
     'aircraft': {'code': '73H'},
     'operating'